[View in Colaboratory](https://colab.research.google.com/github/manan-arya/Analytics_Vidhya_Competition/blob/master/Reference_Submission.ipynb)

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


file_list = drive.ListFile({'q': "'1qErnIcUy-6CaHNrCZbch1sLvthEfsLMb' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: train_NIR5Yl1.csv, id: 1-HHopZhoSruOdGlw8RvPkT4hSz7dZ0ZW
title: test_8i3B3FC.csv, id: 1l1oQujyMoR09N18pkBPdw5tJzX7vMJ39
title: train_HNzkrPW.zip, id: 18w5yFqkAgqhWXmzPLoSL5zLoZGVt9FXV
title: test_Rj9YEaI.csv, id: 19_gyxB5QOj5ak-tGgER23qGEz06lhY91


In [0]:
train_enignma = drive.CreateFile({'id': '1-HHopZhoSruOdGlw8RvPkT4hSz7dZ0ZW'})
train_enignma.GetContentFile('train_en.csv')

test_enignma = drive.CreateFile({'id': '1l1oQujyMoR09N18pkBPdw5tJzX7vMJ39'})
test_enignma.GetContentFile('test_en.csv')

In [0]:
!ls

adc.json	 image_data   test_faces_found2.csv	     train_en.csv
bbox_train.csv	 models       test_faces_found.csv	     train_HNzkrPW.zip
data.zip	 sample_data  test_faces_found_hog.csv
dlib		 test.csv     test_faces_found-nosample.csv
faces_found.csv  test_en.csv  train.csv


In [0]:
import pandas as pd 
import numpy as np 

data = pd.read_csv('train_en.csv')
data.drop('ID', axis=1, inplace=True)
data.head()

,Tag,Reputation,Answers,Username,Views,Upvotes
0,a,3942.0,2.0,155623,7855.0,42.0
1,a,26046.0,12.0,21781,55801.0,1175.0
2,c,1358.0,4.0,56177,8067.0,60.0
3,a,264.0,3.0,168793,27064.0,9.0
4,c,4271.0,4.0,112223,13986.0,83.0


In [0]:
data.groupby('Tag')['Upvotes'].sum()

Tag
a    10030204.0
c    23735061.0
h     7911376.0
i     3803327.0
j    39031622.0
o     1631440.0
p    17681927.0
r      983257.0
s     5528283.0
x     1055459.0
Name: Upvotes, dtype: float64

In [0]:
data = pd.concat([data,pd.get_dummies(data['Tag'], drop_first=True)], axis=1); data.head()

,Tag,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,a,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,a,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,c,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,a,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,c,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [0]:
data.drop('Tag', axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330045 entries, 0 to 330044
Data columns (total 14 columns):
Reputation    330045 non-null float64
Answers       330045 non-null float64
Username      330045 non-null int64
Views         330045 non-null float64
Upvotes       330045 non-null float64
c             330045 non-null uint8
h             330045 non-null uint8
i             330045 non-null uint8
j             330045 non-null uint8
o             330045 non-null uint8
p             330045 non-null uint8
r             330045 non-null uint8
s             330045 non-null uint8
x             330045 non-null uint8
dtypes: float64(4), int64(1), uint8(9)
memory usage: 15.4 MB


In [0]:
data.shape

(330045, 7)

In [0]:
from sklearn.model_selection import train_test_split

X = data.drop('Upvotes', axis=1)
y = data['Upvotes']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(264036, 13)
(264036,)
(66009, 13)
(66009,)


In [0]:
import xgboost as xgb

xg_reg = xgb.XGBRegressor(objective='reg:linear', n_estimators=10, seed=123)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test,preds))
print("RMSE: %f" % (rmse))

RMSE: 2617.374961


In [0]:
test = pd.read_csv('test_en.csv')
test_new = test.copy()
print(test_new.head())
test_new.drop("ID", axis=1, inplace=True)

       ID Tag  Reputation  Answers  Username    Views
0  366953   a      5645.0      3.0     50652  33200.0
1   71864   c     24511.0      6.0     37685   2730.0
2  141692   i       927.0      1.0    135293  21167.0
3  316833   i        21.0      6.0    166998  18528.0
4  440445   i      4475.0     10.0     53504  57240.0


In [0]:
test_new = pd.concat([test_new,pd.get_dummies(test_new['Tag'], drop_first=True)], axis=1); test_new.head()


,Tag,Reputation,Answers,Username,Views,c,h,i,j,o,p,r,s,x
0,a,5645.0,3.0,50652,33200.0,0,0,0,0,0,0,0,0,0
1,c,24511.0,6.0,37685,2730.0,1,0,0,0,0,0,0,0,0
2,i,927.0,1.0,135293,21167.0,0,0,1,0,0,0,0,0,0
3,i,21.0,6.0,166998,18528.0,0,0,1,0,0,0,0,0,0
4,i,4475.0,10.0,53504,57240.0,0,0,1,0,0,0,0,0,0


In [0]:
test_new.drop('Tag', axis = 1, inplace=True)
preds = xg_reg.predict(test_new)

submission = {"ID" : list(test["ID"]), "Upvotes":preds}
submission = pd.DataFrame(submission)
submission.head()

,ID,Upvotes
0,366953,122.028908
1,71864,204.079895
2,141692,84.805763
3,316833,84.805763
4,440445,122.028908


In [0]:
submission.to_csv('enigma_sub_2.csv', index=False)
from google.colab import files 
files.download('enigma_sub_2.csv')

## Tuning the XGBoost


,Reputation,Answers,Username,Views,c,h,i,j,o,p,r,s,x
0,3942.0,2.0,155623,7855.0,0,0,0,0,0,0,0,0,0
1,26046.0,12.0,21781,55801.0,0,0,0,0,0,0,0,0,0
2,1358.0,4.0,56177,8067.0,1,0,0,0,0,0,0,0,0
3,264.0,3.0,168793,27064.0,0,0,0,0,0,0,0,0,0
4,4271.0,4.0,112223,13986.0,1,0,0,0,0,0,0,0,0
5,440.0,1.0,23866,5577.0,0,0,0,0,0,0,1,0,0
6,147.0,1.0,10759,17184.0,1,0,0,0,0,0,0,0,0
7,2269.0,2.0,54623,312.0,0,0,0,1,0,0,0,0,0
8,111.0,2.0,172926,53738.0,0,0,0,1,0,0,0,0,0
9,2749.0,1.0,10189,227.0,1,0,0,0,0,0,0,0,0


In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


steps = [('clf', xgb.XGBRegressor())]


clf_param_grid = {
    'clf__max_depth' : [5,10,20],
    'clf__learning_rate' : [0.1,0.01],
    'clf__n_jobs' : [4],
    'clf__n_estimators' : [50, 100,]
    
}


xgbclf = Pipeline(steps)

X_new = X.copy()
# X_new.drop('ID',axis=1,inplace=True)
estimator = GridSearchCV(xgbclf, param_grid=clf_param_grid ,cv =4, verbose=2, scoring="neg_mean_squared_error")
estimator.fit(X_new,y)

print(estimator.best_params_)
import numpy as np 
print(np.sqrt(-1*estimator.best_score_))

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4, total=   6.3s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.6s remaining:    0.0s


[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4, total=   6.9s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4, total=   7.0s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=50, clf__n_jobs=4, total=   7.0s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jobs=4, total=  13.9s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jobs=4, total=  13.5s
[CV] clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jobs=4 
[CV]  clf__learning_rate=0.1, clf__max_depth=5, clf__n_estimators=100, clf__n_jo

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 24.6min finished


{'clf__learning_rate': 0.1, 'clf__max_depth': 5, 'clf__n_estimators': 100, 'clf__n_jobs': 4}
1395.2965873858348


In [0]:
print(estimator.best_params_)
import numpy as np 
print(np.sqrt(-1*estimator.best_score_))

{'clf__learning_rate': 0.1, 'clf__max_depth': 5, 'clf__n_estimators': 100, 'clf__n_jobs': 4}
1395.2965873858348


In [0]:
test = pd.read_csv('test_en.csv')
test_new = test.copy()
print(test_new.head())
test_new.drop("ID", axis=1, inplace=True)

test_new = pd.concat([test_new,pd.get_dummies(test_new['Tag'], drop_first=True)], axis=1); test_new.head()



test_new.drop('Tag', axis = 1, inplace=True)
preds = estimator.predict(test_new)

submission = {"ID" : list(test["ID"]), "Upvotes":preds}
submission = pd.DataFrame(submission)
submission.head()

       ID Tag  Reputation  Answers  Username    Views
0  366953   a      5645.0      3.0     50652  33200.0
1   71864   c     24511.0      6.0     37685   2730.0
2  141692   i       927.0      1.0    135293  21167.0
3  316833   i        21.0      6.0    166998  18528.0
4  440445   i      4475.0     10.0     53504  57240.0


,ID,Upvotes
0,366953,226.278915
1,71864,118.688744
2,141692,45.328152
3,316833,44.740879
4,440445,273.033539


In [0]:
submission.to_csv('enigma_sub_3.csv', index=False)
from google.colab import files 
files.download('enigma_sub_3.csv')